In [1]:
import carla
import random
import time
import math
from Common import spawn_camera, spawn_vehicle, preprocess_image, follow_curve
import cv2
import numpy as np

# Connessione al simulatore CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
# Ottenere il mondo di simulazione
world = client.get_world()

In [6]:
# Selezioniamo un veicolo dal mondo o creiamone uno
vehicle = spawn_vehicle(world=world)

# Aggiungi una telecamera per monitorare il comportamento
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.5))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Funzione per seguire la curva
def follow_curve(vehicle, curve_points, speed=30):
    # Impostiamo la velocità di base
    throttle = 0.5
    brake = 0.0
    steering = 0.0
    
    # Iteriamo sui punti della curva
    for point in curve_points:
        # Calcoliamo la direzione tra il veicolo e il punto successivo
        dx = point[0] - vehicle.get_location().x
        dy = point[1] - vehicle.get_location().y
        angle_to_target = math.atan2(dy, dx)  # Angolo tra il veicolo e il punto
        
        # Otteniamo la differenza tra l'angolo corrente e l'angolo da percorrere
        angle_difference = angle_to_target - vehicle.get_transform().rotation.yaw
        if angle_difference > math.pi:
            angle_difference -= 2 * math.pi
        elif angle_difference < -math.pi:
            angle_difference += 2 * math.pi
        
        # Impostiamo la sterzata in base all'angolo
        steering = max(-1.0, min(1.0, angle_difference / math.pi))
        
        # Impostiamo la velocità (accelera o frena)
        if abs(angle_difference) < 0.1:  # Se l'angolo è molto piccolo, vai dritto
            throttle = 0.7
        else:  # Altrimenti riduci la velocità
            throttle = 0.3
        
        vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steering, brake=brake, reverse=True))
        
        # Aggiorna il mondo e attendi un po'
        world.tick()
        time.sleep(0.1)

# Definiamo una curva (serie di punti)
# Crea un'immagine in scala di grigi
image = cv2.imread("output/369908.png")

# Definizione dei punti
p0 = (0, 300)       # Punto iniziale
p1 = (400, 300)     # Punto di controllo
p2 = (400, 600)     # Punto finale

# Funzione per calcolare i punti della curva Bézier
def bezier_quadratica(t, p0, p1, p2):
    x = int((1 - t)**2 * p0[0] + 2 * (1 - t) * t * p1[0] + t**2 * p2[0])
    y = int((1 - t)**2 * p0[1] + 2 * (1 - t) * t * p1[1] + t**2 * p2[1])
    return (x, y)

# Genera i punti della curva
num_points = 100
curve_points = [bezier_quadratica(t, p0, p1, p2) for t in np.linspace(0, 1, num_points)]

# Fai seguire al veicolo la curva
follow_curve(vehicle, curve_points)

# Dopo aver fatto seguire la curva al veicolo, puoi fermarlo
vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))

# Chiudere il simulatore
camera.destroy()
vehicle.destroy()


True

In [2]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]

spawn_point = carla.Transform(carla.Location(-1, -40, 2), carla.Rotation(yaw=-90))

vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))

vehicle.set_autopilot(False)

image = cv2.imread("output/369908.png")

# Definizione dei punti
p0 = (0, 300)       # Punto iniziale
p1 = (400, 300)     # Punto di controllo
p2 = (400, 600)     # Punto finale

# Funzione per calcolare i punti della curva Bézier
def bezier_quadratica(t, p0, p1, p2):
    x = int((1 - t)**2 * p0[0] + 2 * (1 - t) * t * p1[0] + t**2 * p2[0])
    y = int((1 - t)**2 * p0[1] + 2 * (1 - t) * t * p1[1] + t**2 * p2[1])
    return (x, y)

# Genera i punti della curva
num_points = 100
curve_points = [bezier_quadratica(t, p0, p1, p2) for t in np.linspace(0, 1, num_points)]

In [3]:
video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    video_output = np.array(preprocess_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))))

camera.listen(lambda image: camera_callback(image))

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True
global vehicle
try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break

        follow_curve(world, vehicle, curve_points)
        cv2.imshow('RGB Camera', video_output)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()

KeyboardInterrupt: 